# **Install preprocessor**

In [ ]:
!pip install tweet-preprocessor

# **Load libraries**

In [ ]:
import pandas as pd
import re
import preprocessor as p
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


# **Read the dataset**

In [ ]:
import pandas as pd

#creating a dataframe for our dataset "subject_questions"
df = pd.read_csv("subjects-questions.csv")
df=df.head(7000)
#printing first 5 rows of our dataset/dataframe
df.head()

,eng,Subject
0,An anti-forest measure is\nA. Afforestation\nB...,Biology
1,"Among the following organic acids, the acid pr...",Chemistry
2,If the area of two similar triangles are equal...,Maths
3,"In recent year, there has been a growing\nconc...",Biology
4,Which of the following statement\nregarding tr...,Physics


In [ ]:
df.shape

(7000, 2)

# **Transform subject labels by numerical values (cluster number)**

In [ ]:
unique_values = df['Subject'].unique()
print(unique_values)

['Biology' 'Chemistry' 'Maths' 'Physics']


In [ ]:
#encoding labels

df['Subject']=df['Subject'].map({'Biology':0,
                             'Chemistry':1,
                             'Physics':2,
                                 'Maths':3})

In [ ]:
df.head()

,eng,Subject
0,An anti-forest measure is\nA. Afforestation\nB...,0
1,"Among the following organic acids, the acid pr...",1
2,If the area of two similar triangles are equal...,3
3,"In recent year, there has been a growing\nconc...",0
4,Which of the following statement\nregarding tr...,2


# **Clean the dataFrame**

In [ ]:
def clean_tweets(df):
    # punctuations we want to be replaced
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
    REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")
    corpus = []
    for line in df:
        # clean data by sending it to tweet_processor
        sentence = p.clean(line)
        # remove puctuation
        sentence = REPLACE_NO_SPACE.sub("", sentence.lower()) # convert all tweets to lower cases
        sentence = REPLACE_WITH_SPACE.sub(" ", sentence)
        corpus.append(sentence)
    return corpus

In [ ]:
df['eng_clean'] = clean_tweets(df['eng'])
df.shape

(7000, 3)

In [ ]:
df['eng_clean']

0       an anti forest measure isa afforestationb sele...
1       among the following organic acids the acid pre...
2       if the area of two similar triangles are equal...
3       in recent year there has been a growingconcern...
4       which of the following statementregarding tran...
                              ...                        
6995    as per brauns principle yield of ammonia will ...
6996    when spheres are raised to the sametemperature...
6997    what is the smallest positive numbergreater th...
6998    the strongest attractive forces area electrost...
6999    the wavelength of a matter wave isgiven bya he...
Name: eng_clean, Length: 7000, dtype: object

# **Corpus**

In [ ]:
corpus = list(df['eng_clean'])

# **Word Embeddings using Tokenizer**

In [ ]:
from keras.preprocessing import sequence
from keras.preprocessing import text
# Create and fit the tokenizer
tokenizer = text.Tokenizer(num_words=2000)
tokenizer.fit_on_texts(corpus)

# Convert sequences to numerical sequences
corpus_embeddings = tokenizer.texts_to_sequences(corpus)

import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences

corpus_embeddings = pad_sequences(corpus_embeddings, maxlen=1000)


#**Agglomerative Clustering**

In [37]:
from sklearn.cluster import AgglomerativeClustering

num_clusters = 4
clustering_model = AgglomerativeClustering(n_clusters=num_clusters)
cluster_assignment = clustering_model.fit_predict(corpus_embeddings)


# **Add clusters predicted values in the dataFrame**:

In [38]:
df['cluster']=cluster_assignment
df.head(10)

,eng,Subject,eng_clean,cluster
0,An anti-forest measure is\nA. Afforestation\nB...,0,an anti forest measure isa afforestationb sele...,0
1,"Among the following organic acids, the acid pr...",1,among the following organic acids the acid pre...,3
2,If the area of two similar triangles are equal...,3,if the area of two similar triangles are equal...,0
3,"In recent year, there has been a growing\nconc...",0,in recent year there has been a growingconcern...,2
4,Which of the following statement\nregarding tr...,2,which of the following statementregarding tran...,1
5,Fern plants reproduce by\nA. Seeds\nB. Spores\...,0,fern plants reproduce bya seedsb sporesc layin...,0
6,Electric current flows through:\nA. a conducto...,2,electric current flows througha a conductorb a...,0
7,The sides of a right angled triangle are in A....,3,the sides of a right angled triangle are in ap...,2
8,If the mass of a body is \( M \) on the\nsurfa...,2,if the mass of a body is \ m \ on thesurface o...,0
9,A particle of mass \( m \) is made to move\nwi...,2,a particle of mass \ m \ is made to movewith u...,2


# **Mapping predicted clusters numbers**



In [39]:
import numpy as np

def tri_et_index(tableau):
    # Obtenir les indices des valeurs triées en ordre décroissant
    indices_tries = np.flip(np.argsort(tableau))

    return indices_tries

In [40]:
import numpy as np

def map_labels(predicted_labels, true_labels):
    unique_predicted_labels = np.unique(predicted_labels)
    label_mapping = {index: 10 for index in unique_predicted_labels}

    for predicted_label in unique_predicted_labels:
        i=0
        mask = predicted_labels == predicted_label
        cluster_true_labels = true_labels[mask]
        true_label_counts = np.bincount(cluster_true_labels)
        indexs=tri_et_index(true_label_counts)
        most_frequent_true_label=indexs[i]
        l=[label_mapping[label] for label in unique_predicted_labels]

        while most_frequent_true_label in l[:predicted_label]:
            most_frequent_true_label = indexs[i+1]
            i+=1
        label_mapping[predicted_label] = most_frequent_true_label

    return label_mapping

label_mapping = map_labels(cluster_assignment,df["Subject"])


In [41]:
mapped_predicted_labels = np.array([label_mapping[label] for label in cluster_assignment])
unique_values = np.unique(mapped_predicted_labels)
label_mapping

{0: 1, 1: 2, 2: 3, 3: 0}

# **Accuracy**

In [42]:
from sklearn.metrics import accuracy_score
clustering_accuracy = accuracy_score(df["Subject"], mapped_predicted_labels)

clustering_accuracy

0.317

#**Silhouette_score**
The average Silhouette score is also used as an evaluation measure in clustering. The best silhouette score is 1 and the worst is -1. Values close to zero indicate that data points are on the boundary i.e overlapping the clusters.

In [43]:
from sklearn.metrics import silhouette_score

# Assuming 'corpus_embeddings' is the sentence representations and 'cluster_assignment' is the predicted clusters
silhouette_avg = silhouette_score(corpus_embeddings, mapped_predicted_labels)
print("Silhouette Score:", silhouette_avg)

Silhouette Score: -0.1051145146371104



# **F_measure (F1-score)**
 F-measure score, which is a metric commonly used to evaluate the clustering performance. It quantifies the trade-off between precision and recall, providing a single value that represents the overall clustering quality. The higher the F-measure score, the better the clustering performance.



In [44]:
from sklearn.metrics import f1_score

# Calculate F-measure
f_measure = f1_score(df["Subject"],  mapped_predicted_labels, average='weighted')

print("F-measure:", f_measure)

F-measure: 0.32018974131948624
